# Misdirection

Some experiments into how gpt2-small behaves when generating false output.

Semantic structure misdirects the model to produce incorrect output. For example,

> They realise John was two years older than Mary. Mary was born before ..."

In [1]:
import torch
from transformer_lens import HookedTransformer

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")

torch.cuda.empty_cache()
torch.set_grad_enabled(False)

# NBVAL_IGNORE_OUTPUT
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

/Users/richardlayte/Work/layterz/experiments/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model gpt2-small into HookedTransformer


In [8]:
from itertools import permutations, product
import einops
from fancy_einsum import einsum
from rich import print
from IPython.display import HTML
from circuitsvis.attention import attention_heads
from transformer_lens.utils import Slice
from plotly import express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "png"

def calculate_logit_diff(cache):
    pred_tokens = torch.tensor([
        [model.to_single_token(c) for c in completions] 
        for _, completions in cache.prompts
    ]).to(device)

    resid_directions = cache.model.tokens_to_residual_directions(pred_tokens)
    return resid_directions[:, 0] - resid_directions[:, 1]

def calculate_head_contribution(cache, towards, layer=-1, pos_slice=-1):
    per_head_residual = cache.stack_head_results(
        layer=layer, pos_slice=pos_slice,
    )

    per_head_logit_diffs = einsum(
        "... batch d_model, batch d_model -> ...",
        per_head_residual, towards,
    )

    return einops.rearrange(
        per_head_logit_diffs,
        "(layer head_index) -> layer head_index",
        layer=cache.model.cfg.n_layers,
        head_index=cache.model.cfg.n_heads,
    )

def calculate_attention_patterns(cache, heads, component="pattern"):
    patterns = {}
    for head in heads:
        layer = head // cache.model.cfg.n_heads
        label = f'{layer}.{head % cache.model.cfg.n_heads}'
        head_index = head % cache.model.cfg.n_heads
        patterns[label] = cache[component, layer].mean(dim=0)[head_index]
    return patterns

def plot_attention_patterns(patterns):
    return [
        px.imshow(
            pattern.cpu(),
            color_continuous_scale="rdbu",
            color_continuous_midpoint=0.0,
            title=label
        )
        for label, pattern in patterns.items()
    ]

def plot_attention_pattern(cache, l, h, title=None, cmap="rdbu", example_index=0):
    print(f'({l}, {h}) cmap={cmap}, example_index={example_index}')
    pattern = cache['attn', l][:, h].mean(dim=0)
    prompt = cache.prompts[example_index]
    tokens = cache.model.to_str_tokens(prompt[0])
    for y in range(len(tokens)):
        for x in range(len(tokens)):
            if x < y:
                pattern[x, y] = -1
    fig = px.imshow(
        pattern.cpu(),
        width=1200, height=800,
        color_continuous_scale=cmap,
        color_continuous_midpoint=0.0,
        title=title,
    )
    fig.update_coloraxes(showscale=False)
    fig.update_layout(
        # set background color to white
        plot_bgcolor='white',
    )

    fig.update_layout(
        # set background color to white
        plot_bgcolor='white',
        margin=dict(l=10, r=10, t=20, b=0)
    )

    for y in range(len(tokens)):
        resid_stream = cache.accumulated_resid(l, apply_ln=True)
        lnt = cache.model.OV[l, h]
        mlp_in = model.W_in[l]
        mlp_out = model.W_out[l]
        pos_slice = Slice((0, y))

        q = cache['q', l].reshape(-1, len(tokens), cache.model.cfg.n_heads * cache.model.cfg.d_head)
        q = pos_slice.apply(q, dim=-2)
        q_proj = torch.einsum('bik,kj->bij', q, lnt.A)
        q_proj = torch.einsum('bik,kl->bil', q_proj, lnt.B)
        q_mlped = torch.einsum('bik,kj->bij', q_proj, mlp_in)
        q_mlped_out = torch.einsum('bik,kl->bil', q_mlped, mlp_out)
        logits = cache.model.unembed(q_mlped_out)
        q_preds = logits.argmax(dim=-1)
        q_str_preds = cache.model.to_str_tokens(q_preds[0])
        
        k = cache['k', l].reshape(-1, len(tokens), cache.model.cfg.n_heads * cache.model.cfg.d_head)
        k = pos_slice.apply(k, dim=-2)
        z = q * k
        z_proj = torch.einsum('bik,kj->bij', z, lnt.A)
        z_proj = torch.einsum('bik,kl->bil', z_proj, lnt.B)
        z_mlped = torch.einsum('bik,kj->bij', z_proj, mlp_in)
        z_mlped_out = torch.einsum('bik,kl->bil', z_mlped, mlp_out)
        z_logits = cache.model.unembed(z_mlped_out)
        z_preds = z_logits.argmax(dim=-1)
        z_str_preds = cache.model.to_str_tokens(z_preds[0])

        v = cache['v', l].reshape(-1, len(tokens), cache.model.cfg.n_heads * cache.model.cfg.d_head)
        v_proj = torch.einsum('bik,kj->bij', v, lnt.A)
        v_proj = torch.einsum('bik,kl->bil', v_proj, lnt.B)
        v_mlped = torch.einsum('bik,kj->bij', v_proj, mlp_in)
        v_mlped_out = torch.einsum('bik,kl->bil', v_mlped, mlp_out)
        v_logits = cache.model.unembed(v_mlped_out)
        v_preds = v_logits.argmax(dim=-1)
        v_str_preds = cache.model.to_str_tokens(v_preds[0])

        logits = cache.model.unembed(resid_stream[-1])
        pred_tokens = logits.argmax(dim=-1)
        pred_str_tokens = cache.model.to_str_tokens(pred_tokens[0])
        # use white if current cell is dark, black otherwise
        for x in range(y):
            text = z_str_preds[x]
            if text.startswith(" "):
                text = text[1:]
            if text == '<|endoftext|>':
                text = 'EOS'
            color = 'black'
            if pattern[y, x] < 0:
                color = 'white'
            fig.add_annotation(
                x=x, y=y,
                text=text,
                textangle=-45,
                showarrow=False, xshift=-2, yshift=0, font=dict(size=16, color=color),
            )
        fig.add_annotation(
            x=x, y=y,
            text=v_str_preds[y],
            showarrow=False, xshift=-2, yshift=0,
            font=dict(size=16, color='white'),
        )
    
    for x in range(len(tokens)):
        text = tokens[x]
        pred_text = pred_str_tokens[x]
        q_str = 'test'
        if text == '<|endoftext|>':
            text = 'EOS'
        if pred_text == '<|endoftext|>':
            pred_text = 'EOS'
        if q_str == '<|endoftext|>':
            q_str = 'EOS'
        #bottom
        fig.add_annotation(x=x, y=y + 1, text=text, showarrow=False, xshift=0, yshift=0, font=dict(size=16, color='black'))
        #left
        fig.add_annotation(x=-1, y=x, text=text, showarrow=False, xshift=0, yshift=0, font=dict(size=16, color='black'))
        #right
        fig.add_annotation(x=len(tokens), y=x, text=pred_text, showarrow=False, xshift=0, yshift=0, font=dict(size=16, color='black'))
        #top
        fig.add_annotation(x=x, y=-1, text=q_str, textangle=90, showarrow=False, xshift=0, yshift=20, font=dict(size=16, color='black'))

    fig.update_layout(
        width=1200, height=1200,
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False)
    )
    return fig

def visualize_attention_patterns(heads, cache, token_labels=None):
    labels, patterns = [], []
    for head in heads:
        layer = head // cache.model.cfg.n_heads
        head_index = head % cache.model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")
        patterns.append(cache["attn", layer][0, head_index])

    patterns = torch.stack(patterns, dim=0)
    if token_labels is None:
        token_labels = [f"Token {i}" for i in range(cache['attn', 0].shape[-1])]

    return attention_heads(
        attention=patterns, tokens=token_labels, attention_head_names=labels
    ).show_code()

def generate_prompts(templates, names):
    return [
        (prompt.format(S, IO), (IO, S))
        for prompt, (S, IO) in product(templates, permutations(names, 2))
    ]

def plot_grid(figs, rows=1, labels=None, title=None, width=1800):
    cols = len(figs) // rows + (len(figs) % rows > 0)
    labels = labels or [f.layout.title.text for f in figs]
    fig = make_subplots(
        rows=rows, cols=cols, 
        subplot_titles=labels,
    )

    for i, f in enumerate(figs):
        fig.add_trace(f.data[0], row=i // cols + 1, col=i % cols + 1)
        fig.update_yaxes(row=i // cols + 1, col=i % cols + 1, autorange="reversed", visible=False)
        fig.update_xaxes(row=i // cols + 1, col=i % cols + 1, visible=False)

    a = figs[0]
    layout = a.layout
    fig.layout.coloraxis = layout.coloraxis
    fig.layout.width = width
    fig.layout.height = ((width // cols) + 100) * rows
    fig.update_coloraxes(showscale=False)
    fig.update_layout(
        title=dict(text=title, font=dict(size=32), automargin=False, yref="paper"),
    )

    return fig

def print_max_logits(cache, component='resid_post', layer=-1, k=5):
    resid_stream = cache[component, layer]
    resid_stream = cache.apply_ln_to_stack(resid_stream, layer)

    logits = cache.model.unembed(resid_stream)
    logits = cache.apply_ln_to_stack(logits, layer)

    top_pred_tokens = torch.topk(logits, k=k, dim=-1).indices.permute(2, 0, 1)
    tokens = cache.model.to_tokens([p for p, _ in cache.prompts])
    pred_resid_directions = cache.model.tokens_to_residual_directions(top_pred_tokens[:, :, :-1])
    token_resid_directions = cache.model.tokens_to_residual_directions(tokens[:, 1:])
    data = torch.stack([
        (token_resid_directions - token_resid_directions).abs().mean((0, -1)),
        *(pred_resid_directions - token_resid_directions).abs().mean((1, -1)),
    ])
    fig = px.imshow(
        data.cpu(),
        color_continuous_midpoint=-0.5,
        color_continuous_scale="rdbu",
    )

    example_prompt = cache.model.to_str_tokens(cache.prompts[0][0])[1:] + ["..."]
    for x in range(len(example_prompt)):
        fig.add_annotation(x=x, y=0, text=example_prompt[x], showarrow=False, xshift=0, yshift=0, font=dict(size=16, color='white'))
        # use white if current cell is dark, black otherwise
        for i in range(k):
            color = 'black' if x == len(example_prompt) - 1 else 'white'
            fig.add_annotation(
                x=x, y=i + 1, 
                text=cache.model.tokenizer.decode(top_pred_tokens[i, 0, x]), 
                showarrow=False, xshift=0, yshift=0, font=dict(size=16, color=color),
            )

    fig.update_layout(
        width=1200, height=800,
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False)
    )

    fig.update_coloraxes(showscale=False)
    return fig

Create clean and corrupted prompt datasets where the corrupted prompts contradict the initial context. Each set is run against the model so we have access to clean and corrupted cached activations.

In [3]:
names = (" Mary", " John", " Alice", " Bob")
prompts = generate_prompts(
    [
        "{0} was two years older than{1}. Who was born first?{0} was born before",
        "{0} was two years younger than{1}. Who was born last?{0} was born after",
    ],
    names
)

corrupted_prompts = generate_prompts(
    [
        "{0} was two years older than{1}. Who was born first?{1} was born before",
        "{0} was two years younger than{1}. Who was born last?{1} was born after",
    ],
    names
)

promtps_equal_length = all([
    len(model.to_str_tokens(p)) for p, _ in prompts + corrupted_prompts
])

if not promtps_equal_length:
    print("Prompts are not equal length")
len(prompts), len(corrupted_prompts), prompts[0], corrupted_prompts[0]

(24,
 24,
 (' Mary was two years older than John. Who was born first? Mary was born before',
  (' John', ' Mary')),
 (' Mary was two years older than John. Who was born first? John was born before',
  (' John', ' Mary')))

In [4]:
model.remove_all_hook_fns()
_, cache = model.run_with_cache([p for p, _ in prompts])
cache.prompts = prompts

_, corrupted_cache = model.run_with_cache([p for p, _ in corrupted_prompts])
corrupted_cache.prompts = corrupted_prompts

Below prints the tokens against the logit value for each. The generated output is added to the end.

In [ ]:
print_max_logits(cache, k=10)

In [ ]:
print_max_logits(corrupted_cache, k=10)

Interestingly, in the corrupted prompt, we see a slightly increased confidence of the false output (~10% increase).

Looking at the attention head patterns many of the top positive and negative contributors are shared, but there are a few noteable exceptions.

In [ ]:
a = px.imshow(
    calculate_head_contribution(cache, calculate_logit_diff(cache)).cpu(),
    labels={"x": "Head", "y": "Layer"},
    color_continuous_midpoint=0.0,
    color_continuous_scale="rdbu",
)

b = px.imshow(
    calculate_head_contribution(corrupted_cache, calculate_logit_diff(corrupted_cache)).cpu(),
    labels={"x": "Head", "y": "Layer"},
    color_continuous_midpoint=0.0,
    color_continuous_scale="rdbu",
)

plot_grid((a, b), labels=["Clean", "Corrupted"], title='Head contributions to logit diff', width=800)

A couple of the top positive heads (e.g. 10.2, 9.8) are shared, but otherwise the top positive contributors for the clean inputs are the top negative contributors for the corrupted inputs (e.g. 9.9, 10.0, 10.6, etc). Many of these are name-mover heads from the IOI paper.

### Postive head patterns

We can dig into this further by looking at what those attention patterns attend to for each input.

In [ ]:
labels, patterns = [], []
for head in range(144):
    layer = head // cache.model.cfg.n_heads
    head_index = head % cache.model.cfg.n_heads
    labels.append(f"L{layer}H{head_index}")
    patterns.append(cache["attn", layer][0, head_index])

patterns = torch.stack(patterns, dim=0)

In [ ]:
k = 5
logit_diff = calculate_logit_diff(cache)
per_head_logit_diffs = calculate_head_contribution(cache, logit_diff)
heads = torch.topk(per_head_logit_diffs.flatten(), k=k).indices

corrupted_logit_diff = calculate_logit_diff(corrupted_cache)
corrupted_per_head_logit_diffs = calculate_head_contribution(corrupted_cache, corrupted_logit_diff)
corrupted_heads = torch.topk(corrupted_per_head_logit_diffs.flatten(), k=k).indices

patterns = calculate_attention_patterns(cache, heads)
corrupted_patterns = calculate_attention_patterns(corrupted_cache, corrupted_heads)

figs = plot_attention_patterns(patterns)
corrupted_figs = plot_attention_patterns(corrupted_patterns)

display(plot_grid(figs, title=f'Top {k} attention heads for clean prompts'))
plot_grid(corrupted_figs, title=f'Top {k} attention heads for corrupted prompts')

In [ ]:
neg_heads = torch.topk(-per_head_logit_diffs.flatten(), k=k).indices
neg_corrupted_heads = torch.topk(-corrupted_per_head_logit_diffs.flatten(), k=k).indices

neg_patterns = calculate_attention_patterns(cache, neg_heads)
neg_corrupted_patterns = calculate_attention_patterns(corrupted_cache, neg_corrupted_heads)

neg_figs = plot_attention_patterns(neg_patterns)
neg_corrupted_figs = plot_attention_patterns(neg_corrupted_patterns)

display(plot_grid(neg_figs, title=f'Bottom {k} attention heads for clean prompts'))
plot_grid(neg_corrupted_figs, title=f'Bottom {k} attention heads for corrupted prompts')

### All heads

I find it interesting to plot the full set of heads by layer to get a sense of how they react under different inputs. Sometimes it's easier to see that there is some kind of pattern or common elements and work backwards - instead of only looking at heads that are deemed "important" using some metric like logit difference. See the pathological inputs notebook for more on this.

In [ ]:
heads = calculate_attention_patterns(cache, range(144))
corrupted_heads = caheads = calculate_attention_patterns(corrupted_cache, range(144))

display(
    plot_grid(
        plot_attention_patterns(heads),
        rows=12,
    )
)

display(
    plot_grid(
        plot_attention_patterns(corrupted_heads),
        rows=12,
    )
)

## Activation patching

We can use activation patching to replace the output of specific components to understand how they contribute to the overall generation process. 

For example, let's try patching these heads, which demonstrate some interesting differences between the clean and corrupted inputs. 

In [ ]:
_, cache = model.run_with_cache([p[0] for p in prompts])
cache.prompts = prompts
_, corrupted_cache = model.run_with_cache([p[0] for p in corrupted_prompts])
corrupted_cache.prompts = corrupted_prompts

indices = [
    (5, 5),
    (6, 9),
]

indices = [ i * cache.model.cfg.n_heads + j for i, j in indices ]
heads = calculate_attention_patterns(cache, indices, component='attn')
corrupted_heads = caheads = calculate_attention_patterns(corrupted_cache, indices, component='attn')

display(
    plot_grid(
        plot_attention_patterns({
            **{f'{k} (clean)': v for k, v in heads.items()},
            **{f'{k} (corrupted)': v for k, v in corrupted_heads.items()}
        }),
    )
)

In the clean input the heads attend to specific tokens in the first sentence which correspond to the question being asked in the current sentence. In the corrupted inputs, where the model is misdirected to produce a false answer, the heads instead attend to the fullstop marking the end of the first sentence.

Swapping the Q activations at these heads between the clean and corrupted caches produces surprising attention patterns. 5.5 is reconstructed almost precisely in both directions, but 6.9 transforms to a very different pattern. However, that pattern is precisely the same in both cases.

In [ ]:
model.remove_all_hook_fns()
activation = 'attn.hook_q'

def swap_attention(index):
    def hook_fn(t, hook):
        t[:, :, index] = corrupted_cache[f'blocks.{index}.{activation}'][:, :, index]
    return hook_fn

model.add_hook(f'blocks.5.{activation}', swap_attention(5))
model.add_hook(f'blocks.6.{activation}', swap_attention(9))

_, ablated_cache = model.run_with_cache([p[0] for p in prompts])
ablated_cache.prompts = prompts

ablated_heads = calculate_attention_patterns(ablated_cache, indices, component='attn')

display(
    plot_grid(
        plot_attention_patterns({
            **{f'{k} (clean)': v for k, v in heads.items()},
            **{f'{k} (clean ablated)': v for k, v in ablated_heads.items()},
        }),
    )
)

In [ ]:
model.remove_all_hook_fns()
activation = 'attn.hook_q'

def swap_attention(index):
    def hook_fn(t, hook):
        t[:, :, index] = cache[f'blocks.{index}.{activation}'][:, :, index]
    return hook_fn

model.add_hook(f'blocks.5.{activation}', swap_attention(5))
model.add_hook(f'blocks.6.{activation}', swap_attention(9))

_, ablated_cache = model.run_with_cache([p[0] for p in corrupted_prompts])
ablated_cache.prompts = corrupted_prompts

ablated_heads = calculate_attention_patterns(ablated_cache, indices, component='attn')

display(
    plot_grid(
        plot_attention_patterns({
            **{f'{k} (corrupted)': v for k, v in corrupted_heads.items()},
            **{f'{k} (corrupted ablated)': v for k, v in ablated_heads.items()},
        }),
    )
)

In [ ]:
model.remove_all_hook_fns()
_, cache = model.run_with_cache([p[0] for p in prompts])
cache.prompts = prompts

In [ ]:
prompts

In [12]:
l = -1
h = -2
q = cache['q', l].reshape(-1, 18, 12 * 64)
q.shape

torch.Size([24, 18, 768])

In [ ]:
cache['k', l].shape

In [13]:
k = cache['k', l].reshape(-1, 18, 12 * 64)
k.shape

torch.Size([24, 18, 768])

In [ ]:
attention_scores = torch.bmm(q, k.transpose(1, 2))
attention_scores.shape

In [ ]:
scaling_factor = torch.sqrt(torch.tensor(768, dtype=torch.float32))
scaled_attention_scores = attention_scores / scaling_factor
scaled_attention_scores.shape

In [ ]:
z = q + k
z.shape

In [14]:
lnt = cache.model.OV[l, h, :, :]
lnt.A.shape, lnt.B.shape

(torch.Size([768, 64]), torch.Size([64, 768]))

In [ ]:
q_proj= torch.einsum('bik,kj->bij', q, lnt.A)
q_proj.shape

In [ ]:
q_proj_b = torch.einsum('bik,kl->bil', q_proj, lnt.B)
q_proj_b.shape

In [ ]:
logits = cache.model.unembed(q_proj_b)
logits.shape

In [ ]:
pred_tokens = logits.argmax(dim=-1)
pred_tokens.shape

In [15]:
mlp_in = model.W_in[l]
mlp_out = model.W_out[l]

mlp_in.shape, mlp_out.shape

(torch.Size([768, 3072]), torch.Size([3072, 768]))

In [33]:
rm = cache[('resid_post', l)]
rm.shape

torch.Size([24, 18, 768])

In [37]:
v = cache['v', l].reshape(-1, 18, 12 * 64)
v.shape

torch.Size([24, 18, 768])

In [36]:
from transformer_lens.utils import Slice
pos_slice = Slice((0, 4))
vv = pos_slice.apply(v, dim=-2)
vv.shape

torch.Size([24, 4, 768])

In [ ]:
pos_slice = slice(0, 11)
v = v[:, pos_slice, :]
v.shape

In [28]:
v = vv
v_proj = torch.einsum('bik,kj->bij', v, lnt.A)
v_proj = torch.einsum('bik,kl->bil', v_proj, lnt.B)

RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (4) for operand 0 and no ellipsis was given

In [ ]:
v_mlped = torch.einsum('bik,kj->bij', v_proj, mlp_in)
v_mlped_out = torch.einsum('bik,kl->bil', v_mlped, mlp_out)

v_mlped.shape, v_mlped_out.shape

In [ ]:
v_logits = cache.model.unembed(v_mlped_out)
v_logits.shape

In [ ]:
v_preds = v_logits.argmax(dim=-1)
v_str_preds = cache.model.to_str_tokens(v_preds[0])
v_str_preds

In [ ]:
pred_str_tokens = cache.model.to_str_tokens(pred_tokens[0])
print(pred_str_tokens)

In [ ]:
_prompts = [p[:100] for p, _ in prompts]
_, short_cache = model.run_with_cache(_prompts)
corrupted_cache.prompts = _prompts

In [ ]:
interesting = [
    (6, 5),
    (8, 3),
    (10, 8),
    (11, 8),
]

In [9]:
import random

cmap = 'haline'
plot_attention_pattern(
    cache,
    random.randint(0, 11),
    random.randint(0, 11),
    cmap=None,
    example_index=random.randint(0, 24),
)

(5, 4) cmap=None, example_index=11

RuntimeError: einsum(): subscript k has size 768 for operand 1 which does not broadcast with previously seen size 0

In [ ]:
_prompts = [p for p, _ in prompts]
_prompts = [model.to_str_tokens(p) for p in _prompts]
_prompts = [p[:14] for p in _prompts]
_prompts = [''.join(p) for p in _prompts]
_prompts = [(p, ()) for p in _prompts]

_, short_cache = model.run_with_cache([p for p, _ in _prompts])
short_cache.prompts = _prompts

_prompts[:5]

In [ ]:
plot_attention_pattern(cache, 8, 3, cmap=None, example_index=0)

In [ ]:
plot_attention_pattern(cache, 8, 3, cmap=None, example_index=5)

In [ ]:
plot_attention_pattern(cache, 8, 3, cmap=None, example_index=10)

In [ ]:
plot_attention_pattern(short_cache, 8, 3, cmap=None, example_index=1)

In [ ]:
plot_attention_pattern(cache, 5, 5, cmap=cmap)

In [ ]:
plot_attention_pattern(cache, 6, 9, cmap=cmap)